In [3]:
import matplotlib
import tensorflow as tf
from keras.datasets import fashion_mnist

matplotlib.use('TkAgg')

from keras.callbacks import ModelCheckpoint

def train(x_train, y_train, x_test, y_test):
    (x_train, x_valid) = x_train[5000:], x_train[:5000]
    (y_train, y_valid) = y_train[5000:], y_train[:5000]

    # Reshape input data from (28, 28) to (28, 28, 1)
    w, h = 28, 28
    x_train = x_train.reshape(x_train.shape[0], w, h, 1)
    x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
    x_test = x_test.reshape(x_test.shape[0], w, h, 1)

    # One-hot encode the labels
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_valid = tf.keras.utils.to_categorical(y_valid, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    # Print training set shape
    print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

    # Print the number of training, validation, and test datasets
    print(x_train.shape[0], 'train set')
    print(x_valid.shape[0], 'validation set')
    print(x_test.shape[0], 'test set')

    model = tf.keras.Sequential()
    # Must define the input shape in the first layer of the neural network
    model.add(
        tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)

    model.fit(x_train, y_train,
              batch_size=64,
              epochs=10,
              validation_data=(x_valid, y_valid),
              callbacks=[checkpointer])

    model.load_weights('model.weights.best.hdf5')

    # Evaluate the model on test set
    score = model.evaluate(x_test, y_test, verbose=0)

    # Print test accuracy
    print('\n', 'Test accuracy:', score[1])


def main():
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    train(x_train, y_train, x_test, y_test)


In [ ]:
main()